# PARTIE Formulaire 

In [140]:
import ipywidgets as widgets
from IPython.display import display
import ipywidgets as widgets


import praw
from classes.Document import Document 
from classes.Author import Author
from classes.Corpus import Corpus
from classes.Document import RedditDocument, ArxivDocument
import datetime 
import pandas as pd 
import urllib.request
import xmltodict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
#=========================




In [141]:
# Création du corpus
mon_corpus = Corpus(nom="Corpus_article")
#del(Document)
#del(mon_corpus)

In [142]:
import ipywidgets as widgets
from IPython.display import display
# Créer des widgets pour le formulaire Arxiv
theme_arxiv_input = widgets.Text(value='foot', description='Thème Arxiv:')
limit_arxiv_input = widgets.IntSlider(value=10, min=1, max=100, description='Limite Arxiv:')

# Créer des widgets pour le formulaire Reddit
theme_reddit_input = widgets.Text(value='Football', description='Thème Reddit:', layout={'margin': '0 50px 0 0'})
limit_reddit_input = widgets.IntSlider(value=10, min=1, max=100, description='Limite Reddit:')
idclient_input = widgets.Text(value='_jjmAvQmLvyPWeH7mSTYrw', description='ID Client:')
secretclient_input = widgets.Text(value='j7vF0wxXN9VuvOrIri3dt3W4fSvH4w', description='Secret Client:')
user_input = widgets.Text(value='td3', description='Utilisateur:')

# Créer des widgets pour les boutons Afficher, Charger et Enregistrer le corpus
button_afficher = widgets.Button(description='Afficher le Corpus')
# Champ de texte pour le chemin du fichier
champ_chemin_fichier = widgets.Text(description="Chemin du fichier:", placeholder="Entrez le chemin du fichier")
button_charger = widgets.Button(description='Charger le Corpus')
button_enregistrer = widgets.Button(description='Enregistrer le Corpus')

# Créer des widgets pour afficher les résultats
result_output = widgets.Output()

# Créer un bouton pour charger les données
load_data_button = widgets.Button(description='Récuperer les données')


# Fonction appelée lors du clic sur le bouton et qui va recuperer les données Arxiv et Reddit 
def load_data(button):
    #PArtie REDDIT
    global collection  # Assurez-vous de déclarer collection comme une variable globale
    theme = theme_reddit_input.value
    limit = limit_reddit_input.value
    idclient = idclient_input.value
    secretclient = secretclient_input.value
    user = user_input.value

    reddit = praw.Reddit(client_id=idclient, client_secret=secretclient, user_agent=user)
    subr = reddit.subreddit(theme)

    collection = []

    for doc in subr.controversial(limit=limit):
        titre = doc.title.replace("\n", '')
        auteur = str(doc.author)
        date = datetime.datetime.fromtimestamp(doc.created).strftime("%Y/%m/%d")
        url = "https://www.reddit.com/" + doc.permalink
        texte = doc.selftext.replace("\n", "")

        doc_class = Document(titre, auteur, date, url, texte)
        collection.append(doc_class)
        
# ====================================================Partie Arxiv : 
#==========================chargement des données Arxiv en instanciant un objet docyment
    theme_arxiv = theme_arxiv_input.value #Foot
    limit_arxiv = limit_arxiv_input.value
    url = f'http://export.arxiv.org/api/query?search_query=all:{theme_arxiv}&start=0&max_results={limit_arxiv}'
    url_read = urllib.request.urlopen(url).read()
   

    # url_read est un "byte stream" qui a besoin d'être décodé
    data =  url_read.decode()
    dico = xmltodict.parse(data) #xmltodict permet d'obtenir un objet ~JSON
    docs = dico['feed']['entry']
    for doc in docs:
        titre = doc["title"].replace('\n', '')  # On enlève les retours à la ligne
        try:
            authors = ", ".join([a["name"] for a in doc["author"]])  # On fait une liste d'auteurs, séparés par une virgule
        except:
            authors = doc["author"]["name"]  # Si l'auteur est seul, pas besoin de liste
        summary = doc["summary"].replace("\n", "")  # On enlève les retours à la ligne
        date = datetime.datetime.strptime(doc["published"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y/%m/%d")  # Formatage de la date en année/mois/jour avec librairie datetime

        doc_class = Document(titre, authors, date, doc["id"], summary)  # Création du Document
        collection.append(doc_class)  # Ajout du Document à la liste.
        
    # Enlever les doublons en se basant sur tout le contenu
    seen_documents = set()
    unique_collection = []

    for doc in collection:
        doc_tuple = (doc.titre, doc.auteur, doc.date, doc.texte, doc.type)
        if doc_tuple not in seen_documents:
            seen_documents.add(doc_tuple)
            unique_collection.append(doc)

    # Garder uniquement les documents avec plus de 200 caractères dans le texte
    filtered_collection = [doc for doc in unique_collection if len(doc.texte) > 200]

    # Afficher le nombre de documents avant et après le nettoyage
    print(f"Nombre de documents avant le nettoyage : {len(collection)}")
    print(f"Nombre de documents après le nettoyage : {len(filtered_collection)}")

    # Réaffecter la liste filtrée à votre collection
    collection = filtered_collection

    #Rempli le corpus 
    for doc in collection:
        if "reddit" in doc.url.lower():
            doc.type = "Reddit"
        else:
            doc.type = "Arxiv"
        mon_corpus.add(doc)
    
        
    with result_output:
        result_output.clear_output(wait=True)
        print("Données chargées avec succès.")
            


#=================================== Creation du corpus======================= 
## =======================================================================
# Fonctions associées aux boutons
def afficher_corpus(button):
    # Logique pour afficher le corpus
    with result_output:
        result_output.clear_output(wait=True)
        mon_corpus.show() 



def enregistrer_corpus(button):
    mon_corpus.save("corpus1.csv")
    with result_output:
        result_output.clear_output(wait=True)
        print("Corpus enregistré avec succès.")
        
def charger_corpus(button):
    chemin_fichier = champ_chemin_fichier.value
    mon_corpus.load(chemin_fichier)
    
    with result_output:
        result_output.clear_output(wait=True)
        print(f"Corpus chargé avec succès depuis le chemin : {chemin_fichier}")



load_data_button.on_click(load_data)#bouton pour recuperer les données
button_afficher.on_click(afficher_corpus)#bouton pour afficher le corpus
button_charger.on_click(charger_corpus)#bouton pour charger le corpus depuis l'ordinateur
button_enregistrer.on_click(enregistrer_corpus)#bouton pour enregistrer le corpus dans lordinateur 



# Conteneurs pour organiser la mise en page
partie_reddit = widgets.VBox([theme_reddit_input, limit_reddit_input, idclient_input, secretclient_input, user_input])
partie_arxiv = widgets.VBox([theme_arxiv_input, limit_arxiv_input, load_data_button])
boutons_bas = widgets.HBox([button_afficher, champ_chemin_fichier, button_charger, button_enregistrer])

# Affichage final
display(widgets.HBox([partie_reddit, partie_arxiv]), boutons_bas, result_output)







Output()

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Nombre de documents avant le nettoyage : 200
Nombre de documents après le nettoyage : 153


FileNotFoundError: [Errno 2] No such file or directory: ''

# Moteur de  recherche

## Methode  1: matrice tf_idf avec mesure de similarité cos 


In [143]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Fonction pour gérer la recherche avec la similarité cosinus
def effectuer_recherche_cosinus(b):
    mon_corpus.construire_mat_tfidf()
    vocab = mon_corpus.get_vocabulaire()
    mots_clefs_utilisateur = champ_recherche_cosinus.value
    top_n = int(champ_top_n.value)
    vecteur_requete = np.zeros(len(vocab))

    mots_clefs_propres = mon_corpus.nettoyer_texte(mots_clefs_utilisateur)
    for mot in mots_clefs_propres.split():
        if mot in vocab:
            mot_id = vocab[mot]['id'] - 1
            vecteur_requete[mot_id] += 1

    #calcule la similarité

    mat_tfidf = mon_corpus.get_mattdidf()
    similarites = cosine_similarity([vecteur_requete], mat_tfidf)

    indices_tries = np.argsort(similarites[0])[::-1]
    id2doc = mon_corpus.get_id2doc()
    
    # zone de sortie 
    with output_recherche_cosinus:
        # Effacer le contenu précédent
        clear_output(wait=True)
        
        for i in range(top_n):
            indice_document = indices_tries[i]
            score_similarite = similarites[0, indice_document]

            #  indexation
            indice_document += 1

            document = id2doc[indice_document]
            
            # Ajouter d'autres informations
            contenu_html = f"<p>Score de similarité avec le document {document.titre} : {score_similarite}</p>" \
                           f"<p>URL : <a href='{document.url}' target='_blank'>{document.url}</a></p>"\
                           f"<p>Contenu du document : {document.texte}</p>" \
                           f"<p>Auteur : {document.auteur}</p>" \
                           f"<p>Date : {document.date}</p>" \
                           f"<p>Type : {document.type}</p>"\
                           f"<hr>"
            
            # Afficher le contenu avec HTML
            display(HTML(contenu_html))

# ...


# Fonction pour effacer la recherche
def effacer_recherche(b):
    champ_recherche_cosinus.value = ""
    # Effacer la sortie
    with output_recherche_cosinus:
        clear_output(wait=True)

# Champ de recherche pour la similarité cosinus
champ_recherche_cosinus = widgets.Text(description="Mots-clés:", placeholder="Entrez vos mots-clés")

# Champ pour spécifier le nombre de résultats à afficher
champ_top_n = widgets.IntText(description="Top N:", value=5)

# Bouton pour effectuer la recherche avec la similarité cosinus
bouton_recherche_cosinus = widgets.Button(description="Rechercher")
bouton_recherche_cosinus.on_click(effectuer_recherche_cosinus)

# Bouton pour effacer la recherche
bouton_effacer = widgets.Button(description="Effacer les résultats")
bouton_effacer.on_click(effacer_recherche)

# Afficher le formulaire
display(champ_recherche_cosinus, champ_top_n, bouton_recherche_cosinus, bouton_effacer)

# Créer une zone de sortie pour afficher les résultats
output_recherche_cosinus = widgets.Output()
display(output_recherche_cosinus)


Text(value='', description='Mots-clés:', placeholder='Entrez vos mots-clés')

IntText(value=5, description='Top N:')

Button(description='Rechercher', style=ButtonStyle())

Button(description='Effacer les résultats', style=ButtonStyle())

Output()